In [ ]:
# Execute this cell to make this notebook's dependencies available
!pip install -q condacolab
import condacolab
condacolab.install_mambaforge()
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-interchange/ligand_in_water/solvated.pdb
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-interchange/ligand_in_water/README.md
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-interchange/ligand_in_water/tip5p.offxml
!wget -q https://raw.githubusercontent.com/openforcefield/openff-docs/_cookbook_data_main/build/cookbook/colab/openforcefield/openff-interchange/ligand_in_water/environment.yaml
!mamba env update -q --name=base --file=environment.yaml
from google.colab import output
output.enable_custom_widget_manager()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:13
🔁 Restarting kernel...

  Pinned packages:

  - python 3.10.*
  - python_abi 3.10.* *cp310*
  - cudatoolkit 11.8.*


Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... By downloading and using the CUDA Toolkit conda packages, you accept the terms and conditions of the CUDA End User License Agreement (EULA): https://docs.nvidia.com/cuda/eula/index.html

 
For Linux 64, Open MPI is built with CUDA awareness but this support is disabled by default.
To enable it, please set the environment variable OMPI_MCA_opal_cuda_support=true before
launching your MPI processes. Equivalently, you can set the MCA parameter in the command line:
mpiexec --mca opal_cuda_support 1 ...
 
In addition, the UCX support is also built but

# Solvating and equilibrating a ligand in a box of water

In [ ]:
import time

import mdtraj
import nglview
import numpy
import openmm
import openmm.app
import openmm.unit
from openff.toolkit import ForceField, Molecule, Topology
from openff.units import unit
from rich.pretty import pprint

from openff.interchange import Interchange
from openff.interchange.components._packmol import RHOMBIC_DODECAHEDRON, pack_box
from openff.interchange.interop.openmm import to_openmm_positions

## Construct the topology

In this example we'll construct a topology consisting of one ligand in a rhombic dodecahedral box with 2100 water molecules. We'll use a _mapped_ SMILES when creating `Molecule` objects to ensure the atom ordering matches. (Atom ordering is not strictly a part of SMILES and therefore liable to be changed with updates to RDKit.)

This can be extended or modified by i.e.
* Replacing this sample ligand with a different ligand of interest - substitute out the ligand SMILES
* Using a different number of water molecules - substitute out the `2100` used below
* Adding ions or co-solvents into the box - add more `Molecule` object as desired


In [ ]:
ligand = Molecule.from_smiles("C1=CC=C(C(=C1)CC(=O)O)NC2=C(C=CC=C2Cl)Cl")
water = Molecule.from_smiles("O")

There are a few ways to convert the information in this trajectory to an Openff [`Topology`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology) object. Since we already know how many of which molecules we want, we'll use a PACKMOL wrapper shipped with Interchange. The `Topology` object returned by `pack_box` contains the ligand, 1000 copies of water, the box vectors we asked for, and the positions generated by PACKMOL.

In [ ]:
topology = pack_box(
    molecules=[ligand, water],
    number_of_copies=[1, 1000],
    box_vectors=3.5 * RHOMBIC_DODECAHEDRON * unit.nanometer,
)
topology.n_molecules, topology.box_vectors, topology.get_positions().shape

(1001,
 array([[3.5       , 0.        , 0.        ],
        [0.        , 3.5       , 0.        ],
        [1.75      , 1.75      , 2.47487373]]) <Unit('nanometer')>,
 (3030, 3))

The ["Sage"](https://openforcefield.org/community/news/general/sage2.0.0-release/) force field line (version 2.x.x) includes TIP3P  parameters for water, so we don't need to use multiple force fields to parametrize this topology. (One could use a different water model provided they accept the risks of using a different one than the force field was optimized with.)

Note that the "Parsley" (version 1.x.x) line did *not* include TIP3P parameters, so loading in an extra force field was required.

In [ ]:
sage = ForceField("openff-2.0.0.offxml")

From here, we can create an ``Interchange`` object, which stores the results of applying the force field to the topology. Since the `Topology` object contained positions and box vectors, we don't need to set them again - they're already set on the `Interchange` object!

In [ ]:
interchange: Interchange = Interchange.from_smirnoff(
    force_field=sage, topology=topology
)
interchange.topology.n_atoms, interchange.box, interchange.positions.shape

(3030,
 array([[3.5       , 0.        , 0.        ],
        [0.        , 3.5       , 0.        ],
        [1.75      , 1.75      , 2.47487373]]) <Unit('nanometer')>,
 (3030, 3))

Now, we can prepare everything that OpenMM needs to run and report a brief equilibration simulation:
* A [`Simulation`](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation) object containing
  * An `openmm.System`
  * A topology in OpenMM's object model (`openmm.app.Topology`)
  * Positions and box vectors in OpenMM's unit solution (`openmm.unit.Quantity`)
* A barostat, since we want to use NPT dynamics to relax the box size toward equilibrium
* An integrator
* Reporters for the trajectory and simulation data

For convenience, let's wrap some boilerplate code into a function that can be called again later with different inputs.

In [ ]:
def create_simulation(
    interchange: Interchange,
    pdb_stride: int = 500,
    trajectory_name: str = "trajectory_100000.pdb",
) -> openmm.app.Simulation:
    integrator = openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    barostat = openmm.MonteCarloBarostat(
        1.0 * openmm.unit.bar, 293.15 * openmm.unit.kelvin, 25
    )

    simulation = interchange.to_openmm_simulation(
        combine_nonbonded_forces=True,
        integrator=integrator,
    )

    simulation.system.addForce(barostat)

    # https://github.com/openmm/openmm/wiki/Frequently-Asked-Questions#why-does-it-ignore-changes-i-make-to-a-system-or-force
    simulation.context.reinitialize(preserveState=True)

    # https://github.com/openmm/openmm/issues/3736#issuecomment-1217250635
    simulation.minimizeEnergy()

    simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)
    simulation.context.computeVirtualSites()

    pdb_reporter = openmm.app.PDBReporter(trajectory_name, pdb_stride)
    state_data_reporter = openmm.app.StateDataReporter(
        "data.csv",
        10,
        step=True,
        potentialEnergy=True,
        temperature=True,
        density=True,
    )
    simulation.reporters.append(pdb_reporter)
    simulation.reporters.append(state_data_reporter)

    return simulation

In [ ]:
simulation = create_simulation(interchange)

Finally, we can run this simulation. This should take approximately 10-20 seconds on a laptop or small workstation.

Again, let's wrap this up into a function to avoid copy-pasting code.

In [ ]:
def run_simulation(simulation: openmm.app.Simulation, n_steps: int = 100000):
    print("Starting simulation")
    start_time = time.process_time()

    print("Step, volume (nm^3)")

    for step in range(n_steps):
        simulation.step(1)
        if step % 500 == 0:
            box_vectors = simulation.context.getState().getPeriodicBoxVectors()
            print(step, numpy.linalg.det(box_vectors._value).round(3))

    end_time = time.process_time()
    print(f"Elapsed time: {(end_time - start_time):.2f} seconds")

In [ ]:
run_simulation(simulation)

Starting simulation
Step, volume (nm^3)
0 30.317
500 31.058
1000 31.224
1500 31.046
2000 30.802
2500 30.741
3000 30.635
3500 30.891
4000 30.96
4500 31.26
5000 30.577
5500 30.399
6000 30.93
6500 30.747
7000 31.119
7500 31.065
8000 30.575
8500 30.922
9000 30.929
9500 30.561
10000 30.524
10500 30.738
11000 30.713
11500 30.698
12000 30.923
12500 30.854
13000 30.473
13500 30.817
14000 30.619
14500 30.533
15000 30.986
15500 30.766
16000 31.142
16500 30.765
17000 30.904
17500 31.341
18000 30.824
18500 30.53
19000 30.374
19500 30.595
20000 30.636
20500 30.123
21000 30.61
21500 30.47
22000 30.447
22500 30.271
23000 30.612
23500 30.385
24000 30.792
24500 30.609
25000 30.827
25500 30.646
26000 30.869
26500 29.909
27000 30.382
27500 30.308
28000 30.302
28500 30.477
29000 30.533
29500 30.622
30000 30.485
30500 30.659
31000 30.665
31500 30.451
32000 30.361
32500 30.525
33000 30.719
33500 31.1
34000 31.247
34500 30.979
35000 31.111
35500 31.053
36000 30.933
36500 30.587
37000 30.889
37500 30.791
3800

## Appendix: A visualizing the trajectory

If [NGLView](http://nglviewer.org/nglview/latest/) is installed, we can use it and MDTraj to load and visualize the PDB trajectory:

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
view = nglview.show_mdtraj(mdtraj.load("trajectory_100000.pdb"))
view.add_representation("licorice", selection="water")
view

NGLWidget(max_frame=199)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Appendix B: using the OPC water model

The `openff-forcefields` package now includes some common water models. After release 2023.05.1, this includes OPC, a 4-site water model from the Amber community. We can load it by simply passing it to the `ForceField` constructor after Sage. (When loading multiple force fields like this, parameters in each section are appended in order of the files.) We can inspect the virtual site parameter in our new in-memory force field.

In [ ]:
force_field_opc = ForceField("openff_unconstrained-2.0.0.offxml", "opc.offxml")
pprint(force_field_opc["VirtualSites"].parameters[0].to_dict())

{
│   'smirks': '[#1:2]-[#8X2H2+0:1]-[#1:3]',
│   'epsilon': <Quantity(0.0, 'kilocalorie_per_mole')>,
│   'type': 'DivalentLonePair',
│   'match': 'once',
│   'distance': <Quantity(-0.15939833, 'angstrom')>,
│   'outOfPlaneAngle': <Quantity(0.0, 'degree')>,
│   'inPlaneAngle': None,
│   'charge_increment1': <Quantity(0.0, 'elementary_charge')>,
│   'charge_increment2': <Quantity(0.679142, 'elementary_charge')>,
│   'charge_increment3': <Quantity(0.679142, 'elementary_charge')>,
│   'rmin_half': <Quantity(1.0, 'angstrom')>,
│   'name': 'EP'
}

We can also get a rough visualization of a single water molecule including the virtual site.

In [ ]:
water.generate_conformers(n_conformers=1)
view = force_field_opc.create_interchange(water.to_topology()).visualize(
    include_virtual_sites=True
)
view.clear_representations()
view.add_representation("ball+stick", aspectRatio=1.5)
view

NGLWidget()

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

Since we want to use a different force field with the same chemical topology - a ligand in a box of water - we can re-use the same `Topology` object we prepared earlier, re-using the same functions we defined above!

In [ ]:
interchange_opc = Interchange.from_smirnoff(
    force_field=force_field_opc,
    topology=topology,
)

In [ ]:
interchange_opc.to_openmm()

<openmm.openmm.System; proxy of <Swig Object of type 'OpenMM::System *' at 0x7d42d2783330> >

In [ ]:
simulation = create_simulation(interchange_opc, trajectory_name="trajectory_opc.pdb")

run_simulation(simulation)

Starting simulation
Step, volume (nm^3)
0 30.317
500 29.78
1000 29.694
1500 29.311
2000 29.574
2500 29.18
3000 29.163
3500 28.816
4000 28.803
4500 29.043
Elapsed time: 2.09 seconds


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# NGLview likes to infer bonds between virtual sites in ways that look messy,
# but you can flip this to `True` just to ensure they're there
show_virtual_sites = False

trajectory = mdtraj.load("trajectory_opc.pdb")

if show_virtual_sites:
    view = nglview.show_mdtraj(trajectory)
else:
    import numpy

    trajectory = mdtraj.load("trajectory_opc.pdb")

    atom_indices = numpy.where(
        [a.element.mass > 0.0 for a in trajectory[0].topology.atoms]
    )[0]

    view = nglview.show_mdtraj(trajectory.atom_slice(atom_indices))

view.add_representation("licorice", selection="water")
view

NGLWidget(max_frame=9)